# India 2022 CheckMARC Mock Report

This is an example document showing of the types of analysis and visualizations that could be prepared for 3M on a regional or country level. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
from dateutil.relativedelta import relativedelta
from pandas.plotting import register_matplotlib_converters
import plotly.graph_objects as go
import plotly.colors as goc
import plotly.figure_factory as ff
import plotly.express as px
register_matplotlib_converters()
import mysql.connector
from mysql.connector import Error

In [2]:
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(user='bla_readonly', password='irradiance',
                              host='db-prod.checkmarc.net', port=3306,
                              database='bluelight_prod')

        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")


In [3]:
# SQL query

q1 = """

SELECT apps_lcu_test.test_date AS 'date', 
apps_clinic.name AS 'clinic', 
CONCAT(apps_user.first_name, ' ', apps_user.last_name) AS 'rep', 
apps_lcu_test.guid AS 'guid', 
apps_group.name AS 'region', 
apps_clinic.country AS 'country',
apps_lcu_test.device_guid AS 'device', 
apps_lcu_test.sensor_type AS 'sensor', 
apps_user_lcu.label AS 'label', 
apps_lcu_test.app_id AS 'app_id',
apps_solar_unit.sticker_id AS 'sticker', 
apps_lcu_test.qa_status AS 'status', 
apps_lcu_test.id AS 'id', 
apps_clinic.city AS 'city', 
apps_clinic.province AS 'province', 
lcu.model_name AS 'brand',
manufacturer.name AS 'manu',
apps_lcu_test.irradiance_result AS 'irradiance',
apps_user_lcu.guid AS 'lcu_guid'

FROM apps_lcu_test

JOIN apps_user ON apps_lcu_test.user_guid = apps_user.guid 
JOIN apps_user_role ON apps_user.guid = apps_user_role.user_guid 
JOIN apps_role ON apps_user_role.role_id = apps_role.id 
JOIN apps_user_lcu ON apps_lcu_test.user_lcu_guid = apps_user_lcu.guid 
JOIN apps_clinic ON apps_user_lcu.clinic_guid = apps_clinic.guid 
JOIN lcu ON apps_user_lcu.lcu_guid = lcu.guid
JOIN manufacturer ON lcu.manufacturer_id = manufacturer.id
LEFT JOIN apps_user_group ON apps_user.guid = apps_user_group.user_guid 
LEFT JOIN apps_group ON apps_user_group.group_guid = apps_group.guid
LEFT JOIN apps_solar_unit ON apps_lcu_test.device_guid = apps_solar_unit.serial_number

WHERE apps_lcu_test.app_id = 2 # Test came from the CheckMARC app AND 
AND apps_user.visibility_id = 0 # Test came from an external user AND 
AND apps_group.name LIKE '3M%'

UNION

SELECT lcu_test.test_datetime AS 'date', 
clinic.name AS 'clinic', 
CONCAT(user.first_name, ' ', user.last_name) AS 'rep',
lcu_test.guid AS 'guid', 
company.name AS 'region', 
clinic.country AS 'country', 
lcu_test.spectrometer_guid AS 'device', 
2 AS 'sensor', 
dentist_lcu.bluelight_id AS 'label', 
3 AS 'app_id', 
lcu_test.spectrometer_guid AS 'sticker',
lcu_test_qcdata.id AS 'status', 
lcu_test.id AS 'id', 
clinic.city AS 'city', 
clinic.province AS 'province', 
lcu.model_name AS 'brand',
manufacturer.name AS 'manu',
lcu_test.average_test_irradiance AS 'irradiance',
dentist_lcu.guid AS 'lcu_guid'

FROM lcu_test
JOIN user ON lcu_test.tested_by_user_id = user.id
JOIN dentist_lcu ON lcu_test.dentist_lcu_guid = dentist_lcu.guid
JOIN dentist ON dentist_lcu.dentist_guid = dentist.guid
JOIN clinic ON dentist.clinic_guid = clinic.guid
JOIN company ON user.company_id = company.id
JOIN lcu ON dentist_lcu.bluelight_lcu_guid = lcu.guid
JOIN lcu_test_qcdata ON lcu_test.id = lcu_test_qcdata.id 
JOIN manufacturer ON lcu.manufacturer_id = manufacturer.id

WHERE company.id != '1' # Test isn't from the Bluelight company
AND clinic.is_demo_clinic = 0
AND clinic.owned_by_company_id != '1'
AND clinic.name NOT LIKE '%test%'
AND clinic.name NOT LIKE '%Test%'
AND clinic.name NOT LIKE '%demo%'
AND clinic.name NOT LIKE '%Demo%'
AND clinic.name NOT LIKE '%training%'
AND clinic.name NOT LIKE '%practice clinic%'
AND clinic.name NOT LIKE '%fake%'
AND clinic.street_address NOT LIKE '%test%'
AND clinic.street_address NOT LIKE '%fake%'

AND company.name LIKE '3M%'

"""

In [4]:
# Save folder
folder = '/Users/johnniven/Documents/Bluelight/Python/India Report/'

# Name of output xlsx file 
save_file = datetime.date.today().strftime('%Y-%m-%d') + '_IndiaReport.xlsx'
save_name = folder + save_file

##########################
# Load data from SQL served
connection = create_connection()
df_raw = pd.read_sql(q1,connection)
##########################

df_raw.fillna('None',inplace=True)

# Make string date columns
df_raw['y'] = df_raw['date'].dt.strftime('%Y')
df_raw['m'] = df_raw['date'].dt.strftime('%m')
df_raw['ym'] = df_raw['date'].dt.strftime('%Y-%m')
df_raw['ymd'] = df_raw['date'].dt.strftime('%Y-%m-%d')

# Make a device type column
conditions = [
(df_raw.app_id == 3),
(df_raw.sensor == 1),
((df_raw.sensor == 2) & (df_raw.app_id == 2))
]

values = ['Legacy', 'Solar','Mobile']
df_raw['devtype'] = np.select(conditions, values)

df_raw = df_raw.convert_dtypes()
df_raw.m = df_raw.m.astype(int)

print('Data loaded')

Connection to MySQL DB successful
Data loaded


In [5]:
# Clean/filter data
def df_cleaner(df):
    
    # Merge solar/spect. sticker columns
    try:
        df['sticker'] = df.sticker.combine_first(df.device)
        df.drop(columns = 'spect',inplace=True)
    except:
        pass
    
    # Remove rep test accounts
    df = df[df.rep.str.contains('Test') == False]
    
    # Manually remove rep's test clinics
    filter_clins = ['Joe Domingo',
    'Alison Brown',
    'Corrie Paulsen',
    'Julie Weber',
    'Gabriele Wehren',
    'Bauerkämper',
    'Claus',
    'Rafaela Oliveira',
    'Hellen Romano',
    'Dr. Rafael Calixto / Dra. Renata Leão',
    'Angel Cerna',
    'Britt Luthren Rep',
    'Manoj Pawar',
    'Maurício Watanabe',
    'Stephano Zerlottini Isaac',
    'Susanne',
    'Sandeep Saini',
    'Daniela Šikulová',
    '3M Patrick Roy',
    'Linda Lavoie-Ferris',
    'Maciel Jr',
    'Martin Dupuis',
    'Scott Hacker',
    'Wolfgang Kislic Nürnberger',
    'Andressa Rodrigues',
    'Debora Longhi',
    'Brock Offerdahl DDS',
    'Julieth Valencia Prueba',
    'Vincenzo De Liddo',
    'clinica dental Dr. Ignacio 3M',
    'Desilu',
    'Wolfgang Kislic',
    'Artis odontologia de precisão',
    'Dr. Adosh Lall', 
    'Pawlowski Dental',
    'van phan',
    'zain',
    '3M Malaysia Sdn Bhd (HQ Sales & Marketing)',
    'frank lafata',
    'Jodi Street']

    df = df[(df.clinic.isin(filter_clins) == False)]

    clinic_filter = ['test','practice clinic','demo','bluelight','toothache', 'lost teeth','3m','fake']
    df = df[df.clinic.str.lower().str.contains('|'.join(clinic_filter)) == False]

    # Jeffery Johnson clinic filter
    jeff_john_filter = ['3M Canada Manufacturing Facility','3M National Equpiment Service Center',
                    'Henry Schein Canada INC','Patterson Dentaire Canada INC','Sinclair Dental CO LTD.','3M Sales Team',
                    'Henry Schein Canada']
    df = df[df.clinic.str.contains('|'.join(jeff_john_filter)) == False]

    # Fix name entered two different ways
    df.rep[df.rep.str.contains('Patil')] = 'Pallavi Patil Patil'
    df.rep[df.rep.str.contains('Priyanka')] = 'Priyanka Sikka'
    df.rep[df.rep.str.contains('Manikya Arabolu')] = 'Manikya Arabolu'
    
    # Remove excess whitespace from names
    df['rep']=df['rep'].apply(lambda x: ' '.join(x.split()))
    
    # Make columns title case
    df['country']=df['country'].apply(lambda x: x.title())
    df['city'] = df['city'].apply(lambda x: x.title())
    df['province'] = df['province'].apply(lambda x: x.title())
    df['clinic'] = df['clinic'].apply(lambda x: x.title())
    df['rep'] = df['rep'].apply(lambda x: x.title())
    
    ### Correct country errors
    
    # Replace DACH region by Germany 
    df[df['3M_region'].str.contains('DACH')].loc['3M_region'] = 'Germany'

    # Correct "Afghanistan"
    for country in ['USA','China','Brasil','Japan','Thailand','New Zealand','Germany','Canada','Spain']:
        df.loc[(df.country=='Afghanistan') & (df['region'].str.contains('DACH')),'country'] = 'Germany'
        df.loc[(df.country=='Algeria') & (df['region'].str.contains('DACH')),'country'] = 'Germany'

        # Correct countries based on the '3M_region' column
        for filt_country in ['Afghanistan','Chad','Sierra Leone','Algeria','Angola','Andorra','Uganda','Nan','nan','None','Uzbekistan','Nauru']:
            df.loc[(df.country==filt_country) & (df['region'].str.contains(country)),'country'] = country

    # Correct country names
    df.country = df.country.replace(to_replace='United States', value='USA')
    df.country = df.country.replace(to_replace='Us', value='USA')
    df.country = df.country.replace(to_replace='Usa', value='USA')
    df.country = df.country.replace(to_replace='Czechia', value='Czech Republic')
    df.country = df.country.replace(to_replace='Brasil', value='Brazil')
    
    ###
    
    # Remove 'XX100'tests not from Brazil
    df = df[df.label.str.upper().str.contains('XX100|ZZ100') == False]

    # Make a "quarter" column
    conditions = [
        (df.m.isin([1,2,3])),
        (df.m.isin([4,5,6])),
        (df.m.isin([7,8,9])),
        (df.m.isin([10,11,12])),
    ]

    values = ['Q1', 'Q2', 'Q3', 'Q4']
    df['quarter'] = np.select(conditions, values)

    # QC status
    df.status.replace({'0':'Not reviewed','1':'Approved','2':'Deleted'},inplace=True)
    
    return df

# Filtered df
df_all = df_cleaner(df_raw)

# Limit to certain country
df_all_in = df_all.query('country == "India"')

# Limit date range
min_date = '2014-01-01'
max_date = '2022-12-31'
df_all_in = df_all_in[(df_all_in.date>min_date) & (df_all_in.date<max_date)]

# Limit to certain device type
# df_all_in = df_all_in[df_all_in.devtype == 'Solar']

 has no region!


## Test and visit summary

In [13]:
# df all each visit with the number of tests and number of lights tested
df_in = df_all_in.groupby(by=['ymd','ym','y','quarter','clinic','rep','city'],as_index=False).agg(tests=('guid','count'),lights=('lcu_guid','nunique')).sort_values(by='ymd',ascending=False)
display(df_in)

# Visits by quarter
df_visits = df_in.groupby(by=['y','quarter'],as_index=False).agg(visits=('clinic','count')).sort_values(by=['y','quarter'],ascending=[False,False])
cm = sns.light_palette("blue", as_cmap=True)

display(df_visits.style.background_gradient(cmap=cm).format(precision=0))

,ymd,ym,y,quarter,clinic,rep,city,tests,lights
864,2022-02-16,2022-02,2022,Q1,Integrated Dental Care & Implant Centre,Satyanarayan Iyer,Dombivli,1,1
863,2022-02-16,2022-02,2022,Q1,32 Intact Dental & Cosmetic Clinic Dr Vinay Dole,Satyanarayan Iyer,Dombivli,1,1
862,2022-02-12,2022-02,2022,Q1,"Kiran Dental & Orthodontic Care,Thane (E)",Satyanarayan Iyer,Thane,1,1
861,2022-02-12,2022-02,2022,Q1,Dr Saagu Tooth Art,Sandeep Saini,Ludhiana,1,1
860,2022-02-10,2022-02,2022,Q1,Dr. Nadkarnis Dental Clinic,Satyanarayan Iyer,Mumbai,2,2
...,...,...,...,...,...,...,...,...,...
4,2017-11-26,2017-11,2017,Q4,City Dental Centre,Pallavi Patil Patil,Aurangabad,6,4
3,2017-11-05,2017-11,2017,Q4,Bibiamena Dental Hospital,Pallavi Patil Patil,Hyderabad,3,2
2,2017-10-06,2017-10,2017,Q4,Perfect Dental Clinc,Priyanka Sikka,Delhi,2,1
1,2017-09-19,2017-09,2017,Q3,Swathi,Swathi Chandran,Bangalore,3,2


,y,quarter,visits
15,2022,Q1,19
14,2021,Q4,55
13,2021,Q3,92
12,2021,Q2,3
11,2021,Q1,112
10,2020,Q1,57
9,2019,Q4,52
8,2019,Q3,17
7,2019,Q2,45
6,2019,Q1,26


In [ ]:
df_all_in.groupby(by=['rep'],as_index=False).agg(
    tests=('guid','count'),
    visits=('clinic','nunique'),
    unique_lights=('lcu_guid','nunique'),
    days_active = ('ymd','nunique'),
    first_test = ('ymd','min'),
    last_test = ('ymd','max')
    )

## Utilization rates

In [ ]:
# Want utilization rates for previous 3 months INCLUDING the current month

df_util = df[(df.ym>(today+relativedelta(months=-3)).strftime("%Y-%m")) & (df.ym <= thisMonth)]
df_dev = pd.crosstab(df_util['device'], df_util['ym']).rename_axis('',axis='columns').reset_index().rename(columns={'device':'Device'})

# sum visits for each device over previous 3 months
df_dev['Sum'] = df_dev.iloc[:,1:].sum(axis=1)
# find devices with at least 3 visits in the last 3 months
devs = df_dev[df_dev.Sum>=3].Device.tolist()

util_all = pd.DataFrame(data={"Dates": [', '.join(df_util.ym.unique())],"Utilization": [len(devs)]})

df_util = df_util[df_util.device.isin(devs)].drop_duplicates('device')

display(util_all)

# Regional utilization
util_reg = df_util.region.value_counts().to_frame().reset_index().rename(columns = {'index':'Region','region':'Utilization'})

display(util_reg)

# Country utilization
util_cnt = df_util.country.value_counts().to_frame().reset_index().rename(columns = {'index':'Country','country':'Utilization'})

# include countries that tested in the last 12 months but not in the previous 3
all_cnts = df[(df.ym>(today+relativedelta(months=-13)).strftime("%Y-%m")) & (df.ym != thisMonth)].country.unique()

util_cnt_zero = pd.DataFrame(data = 0,columns = util_cnt.columns[1:],index = all_cnts).rename_axis('Country').reset_index()

# find countries not in the utiliation data
util_cnt_zero = util_cnt_zero[util_cnt_zero.Country.isin(util_cnt.Country)==False]
util_cnt = util_cnt.append(util_cnt_zero)

display(util_cnt)

In [ ]:
# Summary of this month's utilization

# Get today's date and the date of the first of this month
#today = datetime.date.today().strftime('%Y-%m-%d')
# Manually set the day
today = pd.Timestamp(2022,1,27)

# Get the current %Y-%m and previous %Y-%m
#thisMonth = datetime.date.today().strftime("%Y-%m")
thisMonth = today.strftime("%Y-%m")
lastMonth = (today+relativedelta(months=-1)).strftime("%Y-%m")

df_util = df[(df.ym>(today+relativedelta(months=-3)).strftime("%Y-%m")) & (df.ym <= thisMonth)]
# df_util = df_util.drop_duplicates(['device','ym'])

# util_all = df_util.groupby(by='ym',as_index=False).count().iloc[:,0:2].rename(columns={'ym':'Month','date':'Utilization'})
# util_all.sort_values(by='Month',ascending=False,inplace=True)
# display(util_all)

# Regional utilization
util_dev = pd.crosstab(df_util['device'], df_util['ym']).rename_axis('',axis='columns').reset_index().rename(columns={'device':'Device'})
dates = np.flip(util_dev.columns[1:])
columns = np.append('Device',dates)
util_dev = util_dev.reindex(columns, axis="columns")

# display(util_dev)

# include devices that tested in the last 12 months but not in the previous 3
all_devs = df_all[(df_all.ym>(today+relativedelta(months=-13)).strftime("%Y-%m")) & (df_all.ym < thisMonth)].device.unique()

df_util_all = df[df.device.isin(all_devs)].drop_duplicates(['device','rep'])[['device','region','3M_region','country','rep','devtype']]
df_util_all.rename(columns = {'devtype':'Type'},inplace=True)
# merge reps that have used the same device
device_reps = []
for dev in all_devs:
    device_reps = np.append(device_reps,', '.join(df_util_all[df_util_all.device==dev].drop_duplicates('rep').rep.tolist()))
df_device_reps = pd.DataFrame({'Device':all_devs,'Reps':device_reps})

df_util_all = df_util_all.drop(columns='rep').drop_duplicates('device')
# make columns title case
cols = df_util_all.columns.tolist()
cols = [col_name.title() for col_name in cols]
df_util_all.set_axis(cols,axis=1,inplace=True)

#
util_dev_all = pd.DataFrame(data = None,columns = util_dev.columns[1:],index = all_devs).rename_axis('Device').reset_index()
util_dev_all = util_dev_all.combine_first(util_dev).fillna(0).sort_values(by = 'Device')


df_util_out = df_util_all.merge(df_device_reps,on='Device')
df_util_out = df_util_out.merge(util_dev_all,on='Device')

df_util_out.iloc[:,-3:] = df_util_out.iloc[:,-3:].astype(int)

df_util_out['Sum'] = df_util_out.iloc[:,-3:].sum(axis=1)

# Make a pass/fail column
conditions = [
    (df_util_out.Sum >= 3),
    (df_util_out.Sum < 3)
]

# Pass -> fail column
df_util_out['Utilization'] = np.select(conditions, ['Pass','Fail'])

df_util_out.rename(columns={'Reps':'Rep(s)','3M_Region':'3M Region'},inplace=True)

df_util_out.sort_values(by=['Region','3M Region','Country','Type','Sum'],ascending = [True,True,True,False,False],inplace=True)

display(df_util_out)

del df_util_all, df_device_reps, df_util, all_devs, cols, util_dev

In [ ]:
# active reps by region and month
df_date = df[df.ym>=(today+relativedelta(months=-prev_months)).strftime("%Y-%m")]
df2 = df_date.drop_duplicates(['ym','rep'])
df_reg_rep = pd.crosstab(df2['region'], df2['ym'])
df_reg_rep = df_reg_rep.rename_axis('',axis='columns').reset_index()

# reorder date columns in descending order
dates = np.flip(np.sort(df_date.ym.unique()))
columns = np.append('region',dates)
df_reg_rep = df_reg_rep.reindex(columns, axis="columns")

# add a "unique 2021 reps" column
df_reps = pd.DataFrame(df.drop_duplicates(['rep','region']).region.value_counts()).reset_index()
df_reps.rename(columns={'index':'region','region':'Unique total'},inplace=True)
df_reg_rep = pd.merge(left=df_reg_rep,right=df_reps,left_on='region',right_on='region',how='inner')
# visits by region and month
df_reg_visits = pd.crosstab(df_date['region'], df_date['ym']).rename_axis('',axis='columns').reset_index()
df_reg_visits = df_reg_visits.reindex(columns, axis="columns")
df_reg_visits['Visits'] = df_reg_visits.iloc[:,2:].sum(axis=1)
df_reg_visits.rename(columns={'region':'Region'},inplace=True)
df_reg_visits

# sort
df_reg_rep.sort_values(by='Unique total',ascending=False,inplace=True)
df_reg_rep.rename(columns={'region':'Region'},inplace=True)

# Region visits as % of monthly goal

num_months = len(df_date.ym.unique())

# Find the number of units in each region
reg_units = df_units[df_units.Country.isin(df.region.unique())]
reg_units = reg_units.eval("Goal = Units * 3")

df_reg_visits_norm = pd.merge(df_reg_visits,reg_units,left_on = 'Region', right_on = 'Country', how='outer').drop(columns='Country')
df_reg_visits_norm.iloc[:,1:prev_months+2] = df_reg_visits_norm.apply(lambda x: 100.0*x.iloc[1:prev_months+2]/x['Goal'],axis=1).astype(float).round(2)
df_reg_visits_norm['Yearly goal %'] = df_reg_visits_norm.apply(lambda x: 100.0*x['Visits']/(num_months*3*x['Goal']),axis=1).astype(float).round(2)
df_reg_visits_norm.rename(columns={'Goal':'Monthly goal'},inplace=True)
df_reg_visits_norm.iloc[:,1:prev_months+2] = df_reg_visits_norm.iloc[:,1:prev_months+2].astype(float).round(2)

# tests by month
df_reg_test = pd.crosstab(df_date['region'], df_date['ym'],values = df['tests'],aggfunc = 'sum').round(2).rename_axis('',axis='columns').reset_index()
df_reg_test = df_reg_test.reindex(columns, axis="columns")
df_reg_test['Tests'] = df_reg_test.iloc[:,1:prev_months+2].sum(axis=1)
df_reg_test.rename(columns={'region':'Region'},inplace=True)
df_reg_test.fillna(0,inplace=True)
df_reg_test.iloc[:,1:] = df_reg_test.iloc[:,1:].astype(int)
df_reg_test

cm = sns.light_palette("blue", as_cmap=True)

# Print results
print('Regional reps:')
display(df_reg_rep)#.style.background_gradient(cmap=cm))
# print(df_reg_rep.to_string(index=False))

print('\nRegional visits:')
display(df_reg_visits)#.style.background_gradient(cmap=cm).format(precision=0))
# print(df_reg_visits.to_string(index=False))

print('\nRegional visits as % of goal:')
# print(df_reg_visits_norm.to_string(index=False))
display(df_reg_visits_norm)#.style.background_gradient(cmap=cm).format(precision=0))

print('\nRegional tests:')
# print(df_reg_test.to_string(index=False))
display(df_reg_test)#.style.background_gradient(cmap=cm))

In [ ]:
# Find the number of units in each country
cnt_units = df_units[df_units.Country.isin(df.region.unique()) == False]
cnt_units = cnt_units.eval("Goal = Units * 3")

dates = df_date.ym.unique()
# Make a blank df with columns as dates and rows as pairs of countries
df_visits = pd.DataFrame(data = None,columns = np.flip(np.sort(dates)),index = cnt_units.Country)#.reset_index()

# interate over each country, filling the df_visits frame NaN's using combine_first
for cnt in df_visits.index.tolist():
    df_cnt = df_date[df_date.country.str.contains(cnt)]
    df_cnt = pd.crosstab(df_cnt.region,df_cnt.ym)#.reset_index()
    df_cnt = df_cnt.rename_axis('',axis='columns').reset_index().rename(columns={'region':'Country'})
    df_cnt.Country = cnt
    df_cnt = df_cnt[df_cnt.columns[::-1]]
    df_cnt.set_index('Country',inplace=True)
    df_visits = df_visits.combine_first(df_cnt)

# rorder dates
df_visits = df_visits[df_visits.columns[::-1]]
df_visits.fillna(0,inplace=True)
df_visits.reset_index(inplace=True)

df_visits = cnt_units.merge(df_visits,on='Country')
df_visits.Country=df_visits.Country.apply(lambda x: ', '.join(x.split('|')))
df_visits.iloc[:,1:] = df_visits.iloc[:,1:].astype(int)
df_visits.sort_values(by='Country',inplace=True)
df_visits_norm = df_visits.copy()
df_visits_norm.iloc[:,3:] = df_visits.apply(lambda x: 100*x.iloc[3:]/x['Goal'],axis=1)
df_visits_norm.iloc[:,3:] = df_visits_norm.iloc[:,3:].round(1)
display(df_visits_norm)

# Change df_visits_norm for plotting heatmaps etc.
df_visits_norm.drop(columns=['Units','Goal'],inplace=True)
df_visits_norm.set_index('Country',inplace=True)

col_name = df_visits_norm[df_visits_norm.index.str.contains('Panama')].index.tolist()[0]
df_visits_norm.rename(index={col_name: 'Central America'},inplace=True)
df_visits_norm.reset_index(inplace=True)
display(df_visits_norm)#.style.background_gradient(cmap=cm).format(precision=0)

## Visits, reps, and tests by country

In [ ]:
df_date = df[df.ym>=(today+relativedelta(months=-prev_months)).strftime("%Y-%m")]

# reorder date columns in descending order
dates = np.flip(np.sort(df_date.ym.unique()))
columns = np.append('country',dates)

In [ ]:
# active reps by region and 

df2 = df_date.drop_duplicates(['ym','rep','country'])
df_cnt_rep = pd.crosstab(df2['country'], df2['ym'])
df_cnt_rep = df_cnt_rep.rename_axis('',axis='columns').reset_index()

# reorder date columns in descending order
df_cnt_rep = df_cnt_rep.reindex(columns, axis="columns")

# add a "unique 2021 reps" column
df_reps = pd.DataFrame(df.drop_duplicates(['rep','country']).country.value_counts()).reset_index()
df_reps.rename(columns={'index':'country','country': 'Unique total'},inplace=True)
df_cnt_rep = pd.merge(left=df_cnt_rep,right=df_reps,left_on='country',right_on='country',how='inner')

# sort
df_cnt_rep.sort_values(by='Unique total',ascending=False,inplace=True)
df_cnt_rep.rename(columns={'country':'Country'},inplace=True)
display(df_cnt_rep)#.style.background_gradient(cmap=cm).format(precision=0)

In [ ]:
# visits by country and month
df_cnt_visits = pd.crosstab(df_date['country'], df_date['ym']).rename_axis('',axis='columns').reset_index()
df_cnt_visits = df_cnt_visits.reindex(columns, axis="columns")
df_cnt_visits['Visits'] = df_cnt_visits.iloc[:,1:].sum(axis=1)
df_cnt_visits.rename(columns={'country':'Country'},inplace=True)
df_cnt_visits.sort_values('Visits',ascending=False,inplace=True)
display(df_cnt_visits)#.style.background_gradient(cmap=cm).format(precision=0)

In [ ]:
# tests by month
df_cnt_test = pd.crosstab(df_date['country'], df_date['ym'],values = df['tests'],aggfunc = 'sum').round(2).rename_axis('',axis='columns').reset_index()
df_cnt_test = df_cnt_test.fillna(0)
df_cnt_test = df_cnt_test.reindex(columns, axis="columns")
df_cnt_test['Tests'] = df_cnt_test.iloc[:,1:].sum(axis=1)
df_cnt_test.rename(columns={'country':'Country'},inplace=True)
df_cnt_test.iloc[:,1:] = df_cnt_test.iloc[:,1:].astype(int)
display(df_cnt_test)#.style.background_gradient(cmap=cm).format(precision=0)

## Visits from all reps

In [ ]:
# visits by rep
df_rep_visits = pd.crosstab(df_date['rep'], df_date['ym']).round(2).rename_axis('',axis='columns').reset_index()
df_rep_visits = df_rep_visits.fillna(0)

dates = np.flip(df_rep_visits.columns.tolist()[1:])
columns = np.append('rep',dates)
df_rep_visits = df_rep_visits.reindex(columns, axis="columns")

df_rep_visits['Visits'] = df_rep_visits.iloc[:,1:].sum(axis=1)

# Make a df with each rep's country and region
df_comp = df.drop_duplicates(['country','rep']).copy()
df_info = pd.merge(right=df_comp,left=df_rep_visits,left_on='rep',right_on='rep',how='outer')[['rep','region','country','3M_region']]

# Add country and region data
df_rep_visits = pd.merge(left=df_rep_visits,right=df_info,left_on='rep',right_on='rep',how='inner')
df_rep_visits.rename(columns={'rep':'Rep','country':'Country','region':'Region','3M_region':'3M Region'},inplace = True)
df_rep_visits.iloc[:,1:prev_months+1] = df_rep_visits.iloc[:,1:prev_months+1].astype(int)
df_rep_visits.sort_values(by=['Region','3M Region','Country','Rep'], inplace=True)

# cols = np.flip(np.sort(df_rep_visits.columns.values))

# df_rep_visits = df_rep_visits[cols]

# reorder date columns in descending order
columns = np.append('region',dates)
# df_reg_rep = df_reg_rep.reindex(columns, axis="columns")

display(df_rep_visits)#.style.background_gradient(cmap=cm).format(precision=0)


In [ ]:
with pd.ExcelWriter(save_name, engine="openpyxl") as writer:
    df_util_out.to_excel(writer, sheet_name='Mid-month summary',index=False)
    util_all.to_excel(writer, sheet_name='Overall utilization',index=False)
    util_reg.to_excel(writer, sheet_name='Regional utilization',index=False)
    util_cnt.to_excel(writer, sheet_name='Country utilization',index=False)
    df_rep_visits.to_excel(writer, sheet_name='All reps',index=False)
    df_reg_rep.to_excel(writer, sheet_name='Region reps',index=False)
    df_cnt_rep.to_excel(writer, sheet_name='Country reps',index=False)
    df_reg_visits.to_excel(writer, sheet_name='Region visits',index=False)
    df_reg_visits_norm.to_excel(writer, sheet_name='Region visits %',index=False)
    df_cnt_visits.to_excel(writer, sheet_name='Country visits',index=False)
    df_reg_test.to_excel(writer, sheet_name='Region tests',index=False)
    df_cnt_test.to_excel(writer, sheet_name='Country tests',index=False)
    df_visits.to_excel(writer, sheet_name='Visits by unit',index=False)
    df_visits_norm.to_excel(writer, sheet_name='Visits by units (%)',index=False)

In [ ]:
# visits by rep

with pd.ExcelWriter(save_name, engine="openpyxl",mode='a',if_sheet_exists="replace",startrow = 2) as writer:

    for region in regions:
        df_reg = df_date[df_date.region==region].copy()
        df_rep = pd.crosstab(df_reg['rep'], df_reg['ym']).round(2).rename_axis('',axis='columns')
        df_rep = df_rep[df_rep.columns[::-1]].reset_index()
        df_rep= df_rep.fillna(0)
        df_rep['Visits'] = df_rep.iloc[:,1:].sum(axis=1)
        df_rep.rename(columns={'rep':'Rep name'},inplace=True)
        df_rep.to_excel(writer, sheet_name=region,index=False)

# Visualizations

In [ ]:
# change column order if it isn't already sorted
if (df_reg_rep.columns[-1] != 'Region'):   
    df_reg_rep = df_reg_rep[df_reg_rep.columns[::-1]]
else:
    pass

dates = df_reg_rep.columns[1:-1].values

months = np.char.lstrip(dates.astype(str),'2021')
months = np.char.lstrip(months,'-').astype(int)
months = np.array([calendar.month_name[i] for i in months])

fig = go.Figure()

for i in np.arange(5):
    df_region = df_reg_rep.iloc[i,:]
    region = df_region[-1]
    values = df_region[1:-1].values

    fig.add_trace(go.Bar(x=dates,
                    y=values,
                    name=region,
                    marker_color=goc.sequential.Blues_r[i],
                    marker_line_color = 'black'
                    ))

fig.update_xaxes(
    showgrid=False,
    ticks="outside",
    tickson="boundaries",
    ticklen=5
)

fig.update_layout(
    title_text='CheckMARC active reps (as of ' + today.strftime('%Y-%m-%d') + ')',
    yaxis=dict(
        title='Active Reps',
        titlefont_size=16,
        tickfont_size=14,
    ),
    xaxis=dict(
        title='Date',
        titlefont_size=16,
        tickfont_size=14,
        # nticks = len(df.ym.unique()),
        # ticktext = df.ym.unique(),
        tickformat = '%b-%Y'
    ),
    legend=dict(
        x=.9,
        y=1,
        font=dict(
            size=16,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=0.5  
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0. # gap between bars of the same location coordinate.
)

# adjust figure size
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="white",
)

fig.update_layout(template='plotly_white')

fig.show()

# fig.write_html(save_folder + 'active_reps.html')
fig.write_image(save_folder + 'active_reps.pdf')

In [ ]:
# change column order if it isn't already sorted
if (df_reg_visits.columns[-1] != 'Region'):   
    df_reg_visits = df_reg_rep[df_reg_rep.columns[::-1]]
else:
    pass

dates = df_reg_rep.columns[1:-1].values

months = np.char.lstrip(dates.astype(str),'2021')
months = np.char.lstrip(months,'-').astype(int)
months = np.array([calendar.month_name[i] for i in months])

fig = go.Figure()

for i in np.arange(5):
    df_region = df_reg_visits.iloc[i,:]
    region = df_region[-1]
    values = df_region[1:-1].values

    fig.add_trace(go.Bar(x=dates,
                    y=values,
                    name=region,
                    marker_color=goc.sequential.Blues_r[i],
                    marker_line_color = 'black'
                    ))

fig.update_xaxes(
    showgrid=True,
    ticks="outside",
    tickson="boundaries",
    ticklen=5
)

fig.update_layout(
    title_text='3M CheckMARC visits (as of ' + today.strftime('%Y-%m-%d') + ')',
    yaxis=dict(
        title='Clinic visits',
        titlefont_size=16,
        tickfont_size=14,
    ),
    xaxis=dict(
        title='Date',
        titlefont_size=16,
        tickfont_size=14,
        # nticks = len(df.ym.unique()),
        # ticktext = df.ym.unique(),
        tickformat = '%b-%Y'
    ),
    legend=dict(
        x=.9,
        y=1,
        font=dict(
            size=16,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=0.5  
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0. # gap between bars of the same location coordinate.
)

# adjust figure size
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="white",
)

fig.update_layout(template='plotly_white')

fig.show()

# fig.write_html(save_folder + 'active_reps.html')
fig.write_image(save_folder + 'clinic_visits.pdf')

# Plot of regional visits with table

In [ ]:
for region in regions:

    # Make the df that gets displayed below the plot
    # change column order if it isn't already sorted
    if (df_reg_rep.columns[0] != 'Region'):   
        df_reg_rep = df_reg_rep[df_reg_rep.columns[::-1]]
    else:
        pass

    # number of units
    reg_units = df_units[df_units.Country == region].Units.at[0]
    reg_goal = reg_units*3
    
    # active reps
    reps = df_reg_rep[df_reg_rep.Region == region].iloc[:,0:-1][::-1]
    reps.Region = 'Active reps'

    # visits
    visits = df_reg_visits[df_reg_visits.Region == region].iloc[:,0:-1][::-1]
    visits.Region = 'visits'

    # visits per unit
    visitsper = visits.copy()
    visitsper.iloc[:,1:] = (visitsper.iloc[:,1:]/reg_units).round(1)
    visitsper.Region = 'visits per unit'

    # visits as % of goal
    goal = (df_reg_visits_norm[df_reg_visits_norm.Region == region].iloc[:,0:-4][::-1]).round(1)
    goal.Region = 'visit goal (%)'

    # tests
    tests = df_reg_test[df_reg_test.Region == region].iloc[:,0:-1][::-1]
    tests.Region = 'tests'

    display_data = pd.concat([reps,visits,visitsper,goal,tests],axis=0).rename(columns = {'Region':'Value'}).set_index('Value').T.reset_index()

    # Make columns title case
    cols = display_data.columns.tolist()
    cols[0] = 'Date'
    cols = [col_name.title() for col_name in cols]
    display_data.set_axis(cols,axis=1,inplace=True)

    # Make numbers ints
    display_data['Active Reps'] = display_data['Active Reps'].apply(int)    
    display_data.Visits = display_data.Visits.apply(int)
    display_data.Tests = display_data.Tests.apply(int)

    ##########
    # Plot values
    dates = display_data.Date
    values = display_data.Visits
    
    # Initialize a figure with ff.create_table(table_data)
    colorscale = [[0, '#043653'],[.5, '#CAE2F6'],[1, '#ffffff']]
    colorscale = [[0, '#1EA0F5'],[.5, '#CAE2F6'],[1, '#ffffff']]
    fig = ff.create_table(display_data, height_constant=60,colorscale = colorscale)

    # Make traces for graph
    trace1 = go.Bar(x=dates, y=values, xaxis='x2', yaxis='y2',
                    marker=dict(color='#1EA0F5',line_color='black'),
                    name=region)

    # Make traces for graph
    xmin = (pd.Timestamp(min(dates))+relativedelta(days=-33)).strftime("%Y-%m-%d")
    xmax = (pd.Timestamp(max(dates))+relativedelta(days=+33)).strftime("%Y-%m-%d")

    # Add trace data to figure
    fig.add_traces([trace1])

    fig.add_shape(type='line',
                x0=xmin,
                y0 = reg_goal,
                x1=xmax,
                y1=reg_goal,
                line=dict(color='green',dash = 'dot'),
                xref='x2',
                yref='y2',
                name = 'Monthly visit goal',
    )
            
    # initialize xaxis2 and yaxis2
    fig['layout']['xaxis2'] = {}
    fig['layout']['yaxis2'] = {}

    # Edit layout for subplots
    fig.layout.yaxis.update({'domain': [0, .25]})
    fig.layout.yaxis2.update({'domain': [.35, 1]})

    # The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
    fig.layout.yaxis2.update({'anchor': 'x2'})
    fig.layout.xaxis2.update({'anchor': 'y2'})
    fig.layout.yaxis2.update({'title': 'Visits'})

    fig.update_layout(
                xaxis2=dict(
                title='Date',
                titlefont_size=16,
                tickfont_size=14,
                # nticks = 7,
                # ticktext = dates,
                tickformat = '%b-%Y',
                tickangle = 0
            ),
            yaxis2=dict(
                title='Visits',
                titlefont_size=16,
                tickfont_size=14,
            ),
    )

    # adjust ylimits
    if max(values) > reg_goal:
        ymax = max(values)
        ymax = 50*round(ymax/50)
    else:
        ymax = reg_goal
        ymax = 50*round(ymax/50)

    # Add 'Monthly goal' text
    fig.add_annotation(
    x=(pd.Timestamp(max(dates))+relativedelta(days=-10)).strftime("%Y-%m-%d"),
    y=reg_goal-ymax/20,
    xref="x2",
    yref="y2",
    text="Monthly goal = " + str(reg_goal),
    showarrow=False,
    font=dict(
        size=16,
        color="green"
        )
    )
    
    # Update the margins to add a title and see graph x-labels.
    fig.layout.margin.update({'t':75, 'l':50})
    fig.layout.update({'title': region + ' CheckMARC visits (as of ' + today.strftime('%Y-%m-%d') + ')' +
                        '<br>Number of units = ' + str(reg_units)})

    # adjust figure size
    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        margin=dict(
            l=50,
            r=50,
            b=0,
            t=100,
            pad=0
        ),
        paper_bgcolor="white",
    )

    # correct axis limits to ignore goal line
    xmin = (pd.Timestamp(min(dates))+relativedelta(days=-15)).strftime("%Y-%m-%d")
    xmax = (pd.Timestamp(max(dates))+relativedelta(days=+15)).strftime("%Y-%m-%d")

    fig.update_xaxes(range=[xmin,xmax])

    # update ylimits
    fig.update_yaxes(range=[0, ymax])

    fig.update_layout(template='plotly_white')

    fig.show()
    fig.write_image(save_folder + region + '_breakdown.pdf')
    

## Regional reports

In [ ]:
df_sum = df_all.copy()
df_sum = df_sum[df_sum.y.isin(['2021','2022'])]

df_sum = df_sum.rename(columns = {'country':'Country','name':'Clinic','rep':'Rep','3M_region':'Region','status':'Review status','ymd':'Date','id':'Test ID'})

# Group countries with shared units
df_sum.Country = df_sum.Country.replace(to_replace='Australia', value='Australia-NZ')
df_sum.Country = df_sum.Country.replace(to_replace='New Zealand', value='Australia-NZ')
df_sum.Country = df_sum.Country.replace(to_replace='South Korea', value='Korea')
df_sum.Country = df_sum.Country.replace(to_replace='Germany', value='Germany-Austria-Switzerland')
df_sum.Country = df_sum.Country.replace(to_replace='Austria', value='Germany-Austria-Switzerland')
df_sum.Country = df_sum.Country.replace(to_replace='Switzerland', value='Germany-Austria-Switzerland')
df_sum.Country = df_sum.Country.replace(to_replace='Netherlands', value='Belgium-Netherlands')
df_sum.Country = df_sum.Country.replace(to_replace='Belgium', value='Belgium-Netherlands')
df_sum.Country = df_sum.Country.replace(to_replace='Croatia', value='Croatia-Romania')
df_sum.Country = df_sum.Country.replace(to_replace='Romania', value='Croatia-Romania')
df_sum.Country = df_sum.Country.replace(to_replace='United Kingdom', value='UK-Ireland')
df_sum.Country = df_sum.Country.replace(to_replace='Ireland', value='UK-Ireland')

regions = np.sort(df_all.region.unique())

for region in regions:
    countries = df_sum[df_sum.region == region].Country.unique()
    df_reg = df_sum[df_sum.Country.isin(countries)]

    # Units, visits, and tests summary
    units = df_units[df_units.Country.isin(countries)]
    units = units.eval("Target = Units*3")
    units.rename(columns = {'Target':'Monthly visit goal'},inplace=True)

    tests = pd.crosstab(df_reg.Country,df_reg.ym)
    tests = tests[tests.columns[::-1]]
    tests = tests.reset_index()

    df_visits = df_reg.drop_duplicates(['Clinic','Rep','Date'])
    visits = pd.crosstab(df_reg.Country,df_reg.ym)
    visits = visits[visits.columns[::-1]]
    visits = visits.reset_index()

    output = pd.merge(left = visits, right = tests, left_on = 'Country', right_on = 'Country', suffixes = (' visits',' tests'))
    output = output[np.append('Country',np.flip(np.sort(output.columns[1:].tolist())))]
    output = pd.merge(left = units, right = output, left_on = 'Country', right_on = 'Country')

    # Summary page
    save_file = save_folder + region + '_summary.xlsx'

    with pd.ExcelWriter(save_file, engine="openpyxl", mode= 'w') as writer:
        output.to_excel(writer, sheet_name='Summary',index=False)

    for country in countries:
    
        # Mobile spect. and solar units
       df_mob = df_reg[(df_reg.Country == country) & (df_reg.devtype.isin(['Solar','Mobile']))]
       df_mob = df_mob.sort_values(by='Date',ascending=False)[['Test ID','Clinic','Rep','Date','Region','Country', 'Review status']]

        # Legacy units
       df_leg = df_reg[(df_reg.Country == country) & (df_reg.devtype == 'Legacy')]
       df_leg = df_leg.sort_values(by='Date',ascending=False)[['Test ID','Clinic','Rep','Date','Region','Country', 'Review status']]

    #    Export
       with pd.ExcelWriter(save_file, engine="openpyxl", mode= 'a',if_sheet_exists='replace') as writer:
           df_mob.to_excel(writer, sheet_name=country + ' - M',index=False)
           df_leg.to_excel(writer, sheet_name=country + ' - L',index=False)

## India 2021 summary

In [ ]:
df_in = df[df.country == 'India']

In [ ]:
# This rep messed up what region her tests were
## Is this a bigger issue?
df_raw[(df_raw.region.str.contains('India')) & (df_raw.country != 'India')]

In [ ]:
print(df_raw.drop_duplicates(['country','region'])[['country','region']].to_string())

Afghanistan                                3M USA - Western
United States                                  3M Canada East
Singapore                               3M APAC - Vietnam

In [ ]:
df[(df.country=='Germany') & (df['3M_region'].str.contains('DACH')==False)]